<a href="https://colab.research.google.com/github/SahilCarterr/FinetuningLLM/blob/main/Bert_base_uncased_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Data Preprocessing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/data_cleaned2jul_witutL.csv')

In [ ]:
df.head()

,question,topic
0,once your ex was your lover will you give them...,Dating and Relationships
1,any,Random
2,what type of guys do u hate on hunch,Dating and Relationships
3,karnapishachini ko kya chahiye,Movies and TV
4,feeling hai woexpress nhi kar,Dating and Relationships


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  1575 non-null   object
 1   topic     1575 non-null   object
dtypes: object(2)
memory usage: 24.7+ KB


##### 1.1 Mapping topics to numbers

In [ ]:
unique_topics = df['topic'].unique()
topic_mapping = {topic: i for i, topic in enumerate(unique_topics)}

# Apply the mapping to the 'topic' column
df['topic'] = df['topic'].map(topic_mapping)

# Display the updated DataFrame and the mapping of unique strings to numbers
print("Mapping of unique strings to numbers:", topic_mapping)

Mapping of unique strings to numbers: {'Dating and Relationships': 0, 'Random': 1, 'Movies and TV': 2, 'Music': 3, 'Technology and Internet': 4, 'Philosophy and Religion': 5, 'technology and Internet': 6, 'Law and Order': 7, 'Health and Wellness': 8, 'Food and Drinks': 9, 'Fashion and Makeup': 10, 'health and Wellness': 11, 'Gaming': 12, 'Sports': 13, 'fashion and Makeup': 14, 'sports': 15, 'Anime': 16, 'anime': 17, 'Finance': 18, 'Travel and Geography': 19, 'Pets and Animals': 20, 'gaming': 21, 'random': 22, 'music': 23}


In [ ]:
df.head()

,question,topic
0,once your ex was your lover will you give them...,0
1,any,1
2,what type of guys do u hate on hunch,0
3,karnapishachini ko kya chahiye,2
4,feeling hai woexpress nhi kar,0


##### 1.2 Splitting the data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test datasets to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Data has been split and saved to train.csv and test.csv.")


Data has been split and saved to train.csv and test.csv.


In [ ]:
!pip -q install pyarrow datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


##### 1.3 Converting to Pytorch dataset

In [ ]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'topic'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'topic'],
        num_rows: 315
    })
})

#### 2. Loading Model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    # Ensure inputs are strings, handle potential None or other non-string types
    questions = [str(q) if q is not None else "" for q in examples['question']]
    return tokenizer(questions,truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1260 [00:00<?, ? examples/s]

Map:   0%|          | 0/315 [00:00<?, ? examples/s]

##### 2.1 Using Tokenized Dataset

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["question"])
tokenized_datasets = tokenized_datasets.rename_column("topic", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 16]),
 'token_type_ids': torch.Size([8, 16]),
 'attention_mask': torch.Size([8, 16])}

##### 2.2 Loading Model from Checkpoint

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=24)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##### 2.3 Checking Model output shape

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(3.2043, grad_fn=<NllLossBackward0>) torch.Size([8, 24])


##### 2.2 Initializing Optimizer

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


##### 2.3 Getting training steps

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

474


#### 3. Training the model

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/474 [00:00<?, ?it/s]

In [ ]:
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)


#### 4. Evaluate the model

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score


true_labels = batch['labels'].cpu().numpy()
predicted_labels = predictions.cpu().numpy()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)


precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")


Accuracy: 0.6667, Precision: 1.0000, Recall: 0.6667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
